In [ ]:
import sys
import os 
import pandas as pd
import geopandas as gpd
from pathlib import Path
import json

In [ ]:
proj_dir = Path("./../../")

In [ ]:
sys.path.insert(0, os.path.abspath(proj_dir / "src"))

import thorr
from thorr.utils import config as cfg
from thorr.utils import database
from thorr.utils import logger

In [ ]:
config = cfg.read_config(config_path=str(proj_dir / ".env/config/thorr_config.ini"), required_sections=['mysql'])
db_config_path = config['mysql']["db_config_path"]

db = database.Connect(str(proj_dir / db_config_path))

In [ ]:
connection = db.connection
cursor = connection.cursor()

In [ ]:
db_name = "thorr_transfer_test"

In [ ]:
# Create database if it doesn't exist
cursor.execute(f"CREATE SCHEMA IF NOT EXISTS `{db_name}` DEFAULT CHARACTER SET utf8mb3")

In [ ]:
# turn off foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

# Create the Basins table
basins_query = """
CREATE TABLE IF NOT EXISTS `Basins` (
  `BasinID` SMALLINT NOT NULL AUTO_INCREMENT,
  `Name` varchar(255) NOT NULL,
  `DrainageAreaSqKm` float DEFAULT NULL COMMENT 'Drainage area of the Basin in square-kilometers',
  `MajorRiverID` MEDIUMINT DEFAULT NULL,
  `geometry` geometry NOT NULL /*!80003 SRID 4326 */,
  PRIMARY KEY (`BasinID`),
  UNIQUE KEY `BasinID_UNIQUE` (`BasinID`),
  KEY `Fk_MajorRiver` (`MajorRiverID`),
  CONSTRAINT `Fk_MajorRiver` FOREIGN KEY (`MajorRiverID`) REFERENCES `Rivers` (`RiverID`) ON DELETE SET NULL ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(basins_query)

# Create the Rivers table
rivers_query = """
CREATE TABLE IF NOT EXISTS `Rivers` (
  `RiverID` MEDIUMINT NOT NULL AUTO_INCREMENT,
  `Name` varchar(255) DEFAULT NULL,
  `LengthKm` float DEFAULT NULL COMMENT 'Length of the river in kilometers',
  `WidthM` float DEFAULT NULL COMMENT 'Width in meters',
  `BasinID` SMALLINT DEFAULT NULL COMMENT 'ID for the basin in which this river lies',
  `geometry` geometry NOT NULL /*!80003 SRID 4326 */,
  PRIMARY KEY (`RiverID`),
  UNIQUE KEY `RiverID_UNIQUE` (`RiverID`),
  KEY `Fk_Basin` (`BasinID`),
  CONSTRAINT `Fk_Basin` FOREIGN KEY (`BasinID`) REFERENCES `Basins` (`BasinID`) ON DELETE SET NULL ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(rivers_query)

# Create the Dams table
dams_query = """
CREATE TABLE IF NOT EXISTS `Dams` (
  `DamID` int NOT NULL AUTO_INCREMENT,
  `Name` varchar(255) NOT NULL,
  `Reservoir` varchar(255) DEFAULT NULL,
  `AltName` varchar(255) DEFAULT NULL,
  `RiverID` MEDIUMINT DEFAULT NULL,
  `BasinID` SMALLINT DEFAULT NULL,
  `Country` varchar(255) DEFAULT NULL,
  `Year` year DEFAULT NULL,
  `AreaSqKm` float DEFAULT NULL,
  `CapacityMCM` float DEFAULT NULL,
  `DepthM` float DEFAULT NULL,
  `ElevationMASL` int DEFAULT NULL,
  `MainUse` varchar(255) DEFAULT NULL,
  `LONG_DD` float DEFAULT NULL,
  `LAT_DD` float DEFAULT NULL,
  `DamGeometry` point /*!80003 SRID 4326 */ DEFAULT NULL COMMENT 'Point geometry for the dam',
  `ReservoirGeometry` polygon /*!80003 SRID 4326 */ DEFAULT NULL COMMENT 'Polygon geometry for the reservoir',
  PRIMARY KEY (`DamID`),
  UNIQUE KEY `DamID_UNIQUE` (`DamID`),
  KEY `Fk_basin_dams` (`BasinID`),
  KEY `Fk_river_dams` (`RiverID`),
  CONSTRAINT `Fk_basin_dams` FOREIGN KEY (`BasinID`) REFERENCES `Basins` (`BasinID`) ON DELETE SET NULL ON UPDATE CASCADE,
  CONSTRAINT `Fk_river_dams` FOREIGN KEY (`RiverID`) REFERENCES `Rivers` (`RiverID`) ON DELETE SET NULL ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(dams_query)

# create the Reaches table
reaches_query = """
CREATE TABLE IF NOT EXISTS `Reaches` (
  `ReachID` int NOT NULL AUTO_INCREMENT,
  `Name` varchar(255) DEFAULT NULL,
  `RiverID` MEDIUMINT DEFAULT NULL,
  `ClimateClass` int DEFAULT NULL COMMENT 'Legend linking the numeric values in the maps to the Köppen-Geiger classes.\nThe RGB colors used in Beck et al. [2018] are provided between parentheses',
  `WidthMin` float DEFAULT NULL COMMENT 'Minimum width (meters)',
  `WidthMean` float DEFAULT NULL COMMENT 'Mean width (meters)',
  `WidthMax` float DEFAULT NULL COMMENT 'Maximum width (meters)',
  `RKm` SMALLINT DEFAULT NULL COMMENT 'Distance from the mouth of the river (km)',
  `geometry` geometry NOT NULL /*!80003 SRID 4326 */,
  PRIMARY KEY (`ReachID`),
  UNIQUE KEY `ReachID_UNIQUE` (`ReachID`),
  KEY `Fk_river` (`RiverID`),
  CONSTRAINT `Fk_river` FOREIGN KEY (`RiverID`) REFERENCES `Rivers` (`RiverID`) ON DELETE CASCADE ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(reaches_query)

# Create the DamData table
dam_data_query = """
CREATE TABLE IF NOT EXISTS `DamData` (
  `ID` int NOT NULL AUTO_INCREMENT,
  `Date` date NOT NULL,
  `DamID` int NOT NULL,
  `WaterTempC` float NOT NULL COMMENT 'Landsat-based water temperature for reservoirs',
  `Mission` varchar(4) DEFAULT NULL COMMENT 'The Landsat satellite mission',
  PRIMARY KEY (`ID`),
  UNIQUE KEY `DamDataID_UNIQUE` (`ID`),
  KEY `Fk_water_temp_dam` (`DamID`),
  CONSTRAINT `Fk_water_temp_dam` FOREIGN KEY (`DamID`) REFERENCES `Dams` (`DamID`) ON DELETE CASCADE ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(dam_data_query)

# Create the ReachData table
query = """
CREATE TABLE IF NOT EXISTS `ReachData` (
  `ID` int NOT NULL AUTO_INCREMENT,
  `Date` date NOT NULL,
  `ReachID` int NOT NULL,
  `LandTempC` float DEFAULT NULL COMMENT 'Landsat-based land temperature on the reach corridor. Unit: degrees Celsius',
  `WaterTempC` float DEFAULT NULL COMMENT 'Landsat-based water temperature along the reach. Unit: degrees Celsius',
  `NDVI` float DEFAULT NULL COMMENT 'Landsat-based land temperature on the reach corridor',
  `Mission` VARCHAR(4) NULL COMMENT 'The Landsat satellite mission',
  `EstTempC` float DEFAULT NULL COMMENT 'Estimated water temperature based on the THORR algorithm',
  PRIMARY KEY (`ID`),
  UNIQUE KEY `ReachLandsatDataID_UNIQUE` (`ID`),
  KEY `Fk_landsat_data_reach` (`ReachID`),
  CONSTRAINT `Fk_landsat_data_reach` FOREIGN KEY (`ReachID`) REFERENCES `Reaches` (`ReachID`) ON DELETE CASCADE ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3
"""
cursor.execute(query)

# turn on foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")